In [4]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import datetime
import matplotlib.pyplot as plt
import pickle

In [5]:
import sys
# setting path
sys.path.append('../')

from ruins.processing import drain_cap

In [6]:
# load primary data
data = xr.load_dataset('../data/hydro_krummh.nc').to_dataframe()
data.head()

,Prec_dissagg_1,Prec_dissagg_2,Prec_dissagg_3,Prec_dissagg_4,Prec_dissagg_5,Prec_dissagg_6,Prec_dissagg_7,Prec_dissagg_8,Prec_dissagg_9,Prec_dissagg_10,...,Prec_dissagg_77,Prec_dissagg_78,Prec_dissagg_79,Prec_dissagg_80,Prec,wl_Knock_Outer,wl_LW,wl_Knock_Inner,Knock_pump_obs,Q_Knock
date,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,NaN,-1325.000000,NaN,NaN,NaN
2011-01-01 01:00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,NaN,-1325.000000,NaN,NaN,NaN
2011-01-01 02:00:00,0.0,0.0,0.0,0.491553,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,-1319.090942,NaN,NaN,NaN
2011-01-01 03:00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,NaN,-1315.000000,NaN,NaN,NaN
2011-01-01 04:00:00,0.0,0.0,0.0,0.038447,0.0,0.240214,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,NaN,-1310.877563,NaN,NaN,NaN


In [13]:
#load cached events
with open('cache/events.pkl', 'rb') as file:
    events = pickle.load(file)
with open('cache/canals.pkl', 'rb') as file:
    canal_par = pickle.load(file)
with open('cache/pump.pkl', 'rb') as file:
    pumpcap_fit = pickle.load(file)
canal_par

array([(1.058, 3158.), (0.967, 1888.), (1.164, 5510.), (1.174, 6180.)],
      dtype={'names':['parexponent','parfactor'], 'formats':['<f8','<f8'], 'offsets':[8,16], 'itemsize':8800})

In [ ]:
result = pd.DataFrame(columns = ['max_store', 'energy_cons', 'rcp', 'SLR','prec_incr','event','canal_para', 'canal_area','forecast_pump', 'maxdh','raindissagg'])
i=0
for RCP in ['rcp26','rcp45','rcp85']:
    print(RCP)
#for RCP in ['rcp26']:
    for ClimateTime in ['2065']:
        if RCP == 'rcp26':
            if ClimateTime == '2010':
                SLRs = [35,122,221]  # SLR in 2030 SSP126, outer range of low/medium confidence
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [154, 432, 848] # SLR in 2090 SSP126, outer range of low/medium confidence
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2]
        if RCP == 'rcp45':
            if ClimateTime == '2010':
                SLRs = [29,126,232]  # SLR in 2030 SSP245
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [249, 522, 918] # SLR in 2090 SSP245
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2]
        if RCP == 'rcp85':
            if ClimateTime == '2010':
                SLRs = [34,125,231]  # SLR in 2030 SSP126, outer range of low/medium confidence
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [379, 730, 1143, 1676] # SLR in 2090 SSP126, lower range of low/medium confidence, upper range: 1143 for medium confidence,1676 for low confidence 
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2, 1.3]
        for prec_increase in prec_increases:
            for slr in SLRs:
                print(slr)
####                for rainshift in [-12,-6,0,6,12]:
                for raindissagg in data.filter(regex='Prec').columns:
                    for event in events:
                        t1 = events[event]
                        t2 = events[event]+datetime.timedelta(days=14)
                        hourly_recharge = data[raindissagg][t1:t2]

                        days = (t2-t1).days
                        hours = days*24 + 1

                        tide = data['wl_Knock_Outer'][t1:t2] + slr     # add SLR to tide water level [mm]

                        hourly_recharge = hourly_recharge.rolling("12h").mean() # smoothen rainfall signal to account for water redistribution within catchment
                        hourly_recharge *= prec_increase  # scale hourly recharge by change in climate forcing

                        for advance_pump in [0,50]:
                            for Canal_area in [4,6]:
#                        for advance_pump in [0]:
#                            for Canal_area in [4]:

                                wig = tide*0     # no wind induced gradient -> this may be used to simulate a local flooding in a subcatchment

                                x = pd.DataFrame.from_dict(
                                    {
                                        'recharge' : hourly_recharge.fillna(0),
                                        'h_tide' : tide.fillna(100),
                                        'wig' : wig.fillna(0),
                                    }
                                )
                                for z in canal_par:
                                    for maxdh in [4500,6000]:     # set maximum pump gradient [mm]

    #                                    if i < breaked:
    #                                        i += 1
    #                                        next
    #                                    else:
        #                                z[1] /= canal_flow_scale
                                        x['h_store'], q_pump, x['h_min'], x['flow_rec'], pump_cost, x['store'] = drain_cap.storage_model(forcing_data = x,
                                                                                                                                         canal_par = z,
                                                                                                                                         v_store = 0,
                                                                                                                                         h_store_target = -1350,
                                                                                                                                         canal_area = Canal_area,
                                                                                                                                         h_forecast_pump = advance_pump,
                                                                                                                                         h_grad_pump_max = maxdh,
                                                                                                                                         pump_par = pumpcap_fit)
        #                                my_dict[RCP+'_slr' + str(slr) + '_prec-inc' + str(prec_increase) + '_event' + event + '_adv-pump' + str(advance_pump) + '_canalarea' + str(Canal_area) + '_canalparameter' + str(z)] = max(x['store'])
                                        i += 1
    ####                                    result.loc[str(i),:] = [max(x['store']), sum(pump_cost), RCP, slr,prec_increase,event,str(z),Canal_area, advance_pump, rainshift]
                                        result.loc[str(i),:] = [max(x['store']), sum(pump_cost), RCP, slr,prec_increase,event,str(z),Canal_area, advance_pump, maxdh, raindissagg]
                                        if (i%100==0):
                                            print(i)
result.to_csv("new_try"+datetime.datetime.now().strftime("%Y%m%d%H%M%S")+".csv")
result

rcp26
154
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
